In [6]:
#l'importation des modules utilisés
import subprocess
import pandas as pd
import os
import csv
from pathlib import Path
import re
import numpy as np
from datetime import datetime

In [7]:
#les chemins d'accès aux fichiers
root_file="C:/Users/14387"

os.chdir(root_file)
actual_dir=Path.cwd()

project_repo = actual_dir/"Apache_Hive_Bug_Prediction_ML_/"
hive_repo = project_repo.parent/"Apache_Hive/"
os.chdir(hive_repo)


In [8]:
#l'extraction de données sur la base du projet de laboratoire

#dictionnaire de release
hive_versions = {
"2.0.0": "21cf6ff3789ed94bdf61587e2e73fb94b1d9304c", 
"2.0.1": "115d225d8d311e1d1fa42e8f103c214b26c209a2",
"2.1.0": "c168af26de36ecb66e606b040c862a222ee2a190",
"2.1.1": "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.2.0" : "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.3.0": "90149de71eecb3374fcee0b876324a3b84b8a93d",
"2.3.1": "d7d96658c233b100754884985860bf40f97264ee",
"2.3.2": "b2e411e8cad0da0af370e2b3e2d0bf9cb32dfe7f",
"2.3.3": "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.3.4": "353c55e22d0b0f4603053a7ef45afb5b7b244b75",
"2.3.5": "de69a0f9d973822895f3ff0744ee3af768118268",
"2.3.6": "db59ec6ecf57dad72588b7593fb516c3e810b5a2",
"2.3.7": "03efbb3db1bbccfd3548fa7fa3f130be081e1cdd",
"2.3.8": "89073a94354f0cc14ec4ae0a43e05aae29276b4d",
"2.3.9": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"2.3.10": "7950967eae9640fcc0aa22f4b6c7906b34281eac",
"3.0.0": "ed95ee90608a73fe08747b5483f1bc5ff14bfa22",
"3.1.0": "65f02d2f99b990cb28fd6a832fa3425042e60a04",
"3.1.1": "4b40f7d2b35df26af773ec138fdf521231792cdf",
"3.1.2": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"3.1.3": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"4.0.0": "8b287869bd75665cdcf7f59b64389b6e33cfbab8",
"4.0.1": "799b5cfd95c13433ec46a8aaa23e0b2f38f24d24",
}

#récupérer les fichiers modifiés qui présentent un bogue
input_file = 'Hive_Modified_Files.csv'

bug_ids = []
versions = []
file_names = []

def normalize_path(path):
    return os.path.normpath(path).lower()

with open(input_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    headers = next(reader, None)
    for row in reader:
        if len(row) < 3:
            continue
        bug_id = row[0].strip()
        version = row[1].strip()
        affected_files = row[2].split(';')
        for affected_file in affected_files:
            affected_file = affected_file.strip()
            if affected_file:  # Ensure the file name is not empty
                bug_ids.append(bug_id)
                versions.append(version)
                file_names.append(affected_file)

output_df = pd.DataFrame({
    'Bug ID': bug_ids,
    'Version': versions,
    'File': file_names
})


df_order = output_df.sort_values(by='Version')

#extraire les versions en ordre
for index, row in df_order.iterrows():
    version = row['Version']
    target_file = row['File']

array_version=df_order.drop_duplicates(subset=['Version'])
versions_loop=array_version['Version']


In [9]:
#fonction d'enregistrement des fichiers
def saveonFile(array,version):
    repo = actual_dir/"Apache_Hive_Bug_Prediction_ML_"/"files_output_new_metrics/"
    os.chdir(repo)
    output_file = "File_%s_newMetrics.csv" % (version)

    try:
        with open(output_file, 'w+', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            for file,version,insertionsV,deletionsV,hCounterV,bugFixCounterV,hCounterB4V,devCounterV,devCounterB4V,avgbtwhDays,avgbtwhDaysPlus,expertiseAvg,expertiseMin,codechangedV,codechangedB4 in array:
                writer.writerow([file, version, insertionsV,deletionsV,hCounterV,bugFixCounterV,hCounterB4V,devCounterV,devCounterB4V,avgbtwhDays,avgbtwhDaysPlus,expertiseAvg,expertiseMin,codechangedV,codechangedB4])
            print(f"Output has been written to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")
    os.chdir(hive_repo)  

#recherche par clé dans le dictionnaire
def searchByKey(key):
    keys = list(hive_versions.keys())
    index = keys.index(key)
    index_found=int(index)
    return index_found

#recherche par index dans le dictionnaire
def searchByIndex(index_found):   
    index_found = int(index_found)
    keys = list(hive_versions.keys()) 
    key_index = keys[index_found] 
    value_index = hive_versions[key_index]
    return value_index

#eremoveduplication
def cleanDuplication(array):
    duplicatedCleaned = set(arr.strip() for arr in array) 
    return duplicatedCleaned

def convertToDateISO(dateString):
    date_str = dateString
    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")
    date_iso = date_obj.isoformat()
    return date_iso

def mediaDateCommits(array):
    if len(array)>2:
        # Convert dates to datetime objects
        commit_dates = [datetime.fromisoformat(date.strip()) for date in array]
        
        # Calcular las diferencias entre fechas consecutivas
        time_differences = [(commit_dates[i] - commit_dates[i + 1]).total_seconds() for i in range(len(commit_dates) - 1)]
        
        # Calculate differences between consecutive dates
        median_time_between_changes = np.median(time_differences)
        
        days, remainder = divmod(median_time_between_changes, 86400)
        return int(days)
    else:
        return 0

def getCommitsByAuthor(authorsArray,hash1,hash2):
    expertArr=[]
    for author in authorsArray:
        commitsDone=commitsAffectedByAutor(author,hash1,hash2)
        expertArr.append(int(commitsDone))

    return expertArr

def calculateAvg(arrayList):
    if len(arrayList) > 2:
        avg = sum(arrayList) / len(arrayList)
        return round(avg)
    else:
        return 0

def calculateMin(arrayList):
    if len(arrayList) > 2:
        minArr = min(arrayList)
        return minArr
    else:
        return 0
       
def commitsAffectedByAutor(authorParm,hash1,hash2):
    author = authorParm
    result = subprocess.run(['git', 'log', '--author', author, f'{hash1}..{hash2}', '--pretty=format:%H'], capture_output=True, text=True)
    commits = result.stdout.strip().split('\n')
    counter_commits = len(commits)
    return counter_commits




In [10]:


for version in versions_loop: 
    print(version,"---------------------------------------------------------------------------------------------------------------")
    array=[]
    fileCounter=0;
    # Definir los títulos de las columnas
    titles = np.array([['File', 'Version', 'Insertion','Deletion','hCounterV','bugFixCounter','hCounterB4V','devCounter','devCounterB4',
                        'avgbtwhDays','avgbtwhDaysPlus','expertiseAvg','expertiseMin','codechangedV','codechangedB4']])
    array=titles
    array_in_version_V=output_df[output_df['Version'] == version]
    dateIndex=int(searchByKey(version))
    firsthash='243e7c1ac39cb7ac8b65c5bc6988f5cc3162f558'
    if version=="2.0.0":
        hash1=firsthash
    else:
        hash1=str(searchByIndex(dateIndex-1))
        
    hash2=str(searchByIndex(dateIndex))
    
    for index, row in array_in_version_V.iterrows():
        version = row['Version']
        files_array = row['File']
        
        for file in files_array.strip().split(','):
            fileCounter+=1;
            file_path = file  # e.g. 'file/index.d.ts' 
            
            #columns insertions and deletations -1 and 2  
            insertionAndDeletioncommand = ["git", "diff", "--shortstat", hash1, hash2, "--", file_path]
            subproc1 = subprocess.run(insertionAndDeletioncommand, capture_output=True, text=True)
            output1 = subproc1.stdout
            print(output1)
            #columns hCounter --3
            hCountercommand= ['git', 'log','--follow', f'{hash1}..{hash2}', '--pretty=format:%h', "--",file_path]
            subproc3 = subprocess.run(hCountercommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            counter3=0
            if subproc3.stdout is not None:
                output3 = subproc3.stdout.splitlines()
                counter3=int(len(output3))

            #bugFixCounter-----4 
            keywords="bugfix\|fix\|resolved\|closed"
            bugFixCountercommand = ["git", "log","--grep="+keywords, f'{hash1}..{hash2}', "--",file_path]
            subproc4 = subprocess.run(bugFixCountercommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            counter4=0
            if subproc4.stdout is not None:
                output4 = subproc4.stdout.splitlines()
                counter4=int(len(output4))

            #hCounterB4------5
            hCounterPluscommand = ['git', 'log','--follow', f'{firsthash}..{hash2}', '--pretty=format:%h', "--",file_path]
            subproc5 = subprocess.run(hCounterPluscommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            counter5=0
            if subproc5.stdout is not None:
                output5 = subproc5.stdout.splitlines()
                counter5=int(len(output5))

            #devCounter-------6
            devCountercommand = ['git', 'log', f'{hash1}..{hash2}', '--pretty=format:%an', "--",file_path]
            subproc6 = subprocess.run(devCountercommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            counter6=0
            if subproc6.stdout is not None:
                output6 = subproc6.stdout.splitlines()
                arrayDevduplication=output6
                arrayDev=cleanDuplication(arrayDevduplication)
                counter6=int(len(arrayDev))
           
            #devCounterPlus--7
            devCounterPluscommand = ['git', 'log', f'{firsthash}..{hash2}', '--pretty=format:%an', "--",file_path]
            subproc7 = subprocess.run(devCounterPluscommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            counter7=0
            if subproc7.stdout is not None:
                output7 = subproc7.stdout.splitlines()
                arrayDevduplicationPlus=output7
                arrayDevPlus=cleanDuplication(arrayDevduplicationPlus)
                counter7=int(len(arrayDevPlus))

            
            match = re.search(r'(\d+) insertions\(\+\), (\d+) deletions\(-\)', output1)
            
            if match:
                insertions = int(match.group(1))
                deletions = int(match.group(2))
                new_line = np.array([[file_path, version, insertions,deletions,counter3,counter4,counter5,counter6,counter7,0,0,
                                      0,0,0,0]]) 
                array = np.append(array,new_line, axis=0)
            else: 
                new_line = np.array([[file_path, version, 0,0,counter3,counter4,counter5,counter6,counter7,0,0,
                                      0,0,0,0]]) 
                array = np.append(array, new_line, axis=0)

            
    saveonFile(array,version)  
 

<>:41: SyntaxWarning: invalid escape sequence '\|'
<>:41: SyntaxWarning: invalid escape sequence '\|'
C:\Users\14387\AppData\Local\Temp\ipykernel_9876\1071392601.py:41: SyntaxWarning: invalid escape sequence '\|'
  keywords="bugfix\|fix\|resolved\|closed"


2.0.0 ---------------------------------------------------------------------------------------------------------------
 1 file changed, 350 insertions(+), 211 deletions(-)


 1 file changed, 52 insertions(+), 10 deletions(-)





 1 file changed, 4 insertions(+), 4 deletions(-)

 1 file changed, 32 insertions(+), 46 deletions(-)

 1 file changed, 1 insertion(+), 1 deletion(-)


 1 file changed, 329 insertions(+), 214 deletions(-)

 1 file changed, 210 insertions(+), 191 deletions(-)

 1 file changed, 54 insertions(+)

 1 file changed, 35 insertions(+), 16 deletions(-)

 1 file changed, 122 insertions(+), 26 deletions(-)


 1 file changed, 838 insertions(+), 281 deletions(-)

 1 file changed, 906 insertions(+), 601 deletions(-)

 1 file changed, 228 insertions(+), 42 deletions(-)

 1 file changed, 927 insertions(+), 193 deletions(-)

 1 file changed, 121 insertions(+), 98 deletions(-)


 1 file changed, 63 insertions(+), 13 deletions(-)

 1 file changed, 12 insertions(+), 2 deletions(-)


Exception in thread Thread-93961 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in

 1 file changed, 441 insertions(+), 208 deletions(-)

 1 file changed, 4194 insertions(+), 2171 deletions(-)

 1 file changed, 241 insertions(+), 19 deletions(-)

 1 file changed, 1562 insertions(+), 870 deletions(-)

 1 file changed, 11 insertions(+), 10 deletions(-)

 1 file changed, 224 insertions(+), 114 deletions(-)

 1 file changed, 3036 insertions(+), 2120 deletions(-)

 1 file changed, 133 insertions(+)

 1 file changed, 70 insertions(+), 34 deletions(-)

 1 file changed, 69 insertions(+), 7 deletions(-)

 1 file changed, 66 insertions(+)

 1 file changed, 67 insertions(+)

 1 file changed, 78 insertions(+)

 1 file changed, 583 insertions(+)

 1 file changed, 61 insertions(+)

 1 file changed, 222 deletions(-)

 1 file changed, 6 insertions(+), 22 deletions(-)

 1 file changed, 549 insertions(+), 2633 deletions(-)

 1 file changed, 1462 insertions(+), 446 deletions(-)

 1 file changed, 11 insertions(+), 1 deletion(-)

 1 file changed, 29 insertions(+)

 1 file changed, 28 inse

Exception in thread Thread-95115 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in

...

 1 file changed, 112 insertions(+), 55 deletions(-)

 1 file changed, 181 insertions(+), 227 deletions(-)

 1 file changed, 8 insertions(+), 22 deletions(-)

 1 file changed, 8 insertions(+), 19 deletions(-)

 1 file changed, 6 insertions(+), 33 deletions(-)

 1 file changed, 34 insertions(+)

 1 file changed, 220 insertions(+), 51 deletions(-)

 1 file changed, 26 insertions(+), 57 deletions(-)

 1 file changed, 3 insertions(+), 2 deletions(-)

 1 file changed, 90 insertions(+)

 1 file changed, 82 insertions(+)

 1 file changed, 364 insertions(+)

 1 file changed, 198 insertions(+)

 1 file changed, 4 insertions(+), 12 deletions(-)

 1 file changed, 2 insertions(+), 1 deletion(-)

 1 file changed, 89 insertions(+), 20 deletions(-)

 1 file changed, 88 insertions(+), 31 deletions(-)

 1 file changed, 183 insertions(+), 142 deletions(-)

 1 file changed, 136 insertions(+), 109 deletions(-)

 1 file changed, 127 insertions(+), 17 deletions(-)

 1 file changed, 145 insertions(+), 47 dele

Exception in thread Thread-98775 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in

 1 file changed, 183 insertions(+), 142 deletions(-)

 1 file changed, 6 insertions(+), 6 deletions(-)

 1 file changed, 245 insertions(+), 66 deletions(-)



 1 file changed, 145 insertions(+), 43 deletions(-)

 1 file changed, 5 insertions(+), 2 deletions(-)

 1 file changed, 109 insertions(+), 244 deletions(-)

 1 file changed, 14 insertions(+), 10 deletions(-)

 1 file changed, 1329 insertions(+), 167 deletions(-)

 1 file changed, 139 insertions(+), 15 deletions(-)

 1 file changed, 658 insertions(+), 123 deletions(-)

 1 file changed, 564 insertions(+), 353 deletions(-)

 1 file changed, 89 insertions(+), 20 deletions(-)

 1 file changed, 73 insertions(+), 20 deletions(-)

 1 file changed, 88 insertions(+), 31 deletions(-)

 1 file changed, 40 insertions(+), 2 deletions(-)

 1 file changed, 5 insertions(+), 2 deletions(-)

 1 file changed, 198 insertions(+), 28 deletions(-)

 1 file changed, 5 insertions(+), 5 deletions(-)

 1 file changed, 14 insertions(+), 10 deletions(-)

 1 f

Exception in thread Thread-101343 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f i

 1 file changed, 87 insertions(+), 33 deletions(-)

 1 file changed, 134 insertions(+), 24 deletions(-)

 1 file changed, 87 insertions(+), 33 deletions(-)

 1 file changed, 134 insertions(+), 24 deletions(-)

 1 file changed, 5 insertions(+), 5 deletions(-)

 1 file changed, 564 insertions(+), 353 deletions(-)


 1 file changed, 1 insertion(+), 1 deletion(-)





 1 file changed, 10 insertions(+), 16 deletions(-)

 1 file changed, 4 insertions(+), 13 deletions(-)

 1 file changed, 23 insertions(+), 4 deletions(-)

 1 file changed, 750 deletions(-)

 1 file changed, 495 deletions(-)


 1 file changed, 6 insertions(+), 15 deletions(-)


 1 file changed, 2 insertions(+), 1 deletion(-)

 1 file changed, 28 insertions(+), 2 deletions(-)

 1 file changed, 1 insertion(+), 1 deletion(-)

 1 file changed, 2 insertions(+), 2 deletions(-)



 1 file changed, 183 insertions(+), 142 deletions(-)

 1 file changed, 965 insertions(+), 401 deletions(-)

 1 file changed, 141 insertions(+), 9 deletions(